The zip codes and neighborhoods were often mis-labeled, so I am using the Census block groups to map to zip and to neighborhoods myself.

In [1]:
import pandas as pd

## Let's start w zips

In [24]:
df1 = pd.read_csv('../data/census-data/zcta_tract_rel_10.txt')
df1.head(2)

,ZCTA5,STATE,COUNTY,TRACT,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,...,TRAREA,TRAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,TRPOPPCT,TRHUPCT,TRAREAPCT,TRAREALANDPCT
0,601,72,1,956300,72001956300,4271,1706,44663250,44572589,18570,...,44924558,44833897,23.00,22.03,26.67,26.74,98.5,98.33,99.42,99.42
1,601,72,1,956400,72001956400,2384,1037,32830481,32492074,18570,...,37782601,37191697,12.84,13.39,19.61,19.50,79.6,80.14,86.89,87.36


In [27]:
df1[df1.TRACT == 981700]

,ZCTA5,STATE,COUNTY,TRACT,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,...,TRAREA,TRAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,TRPOPPCT,TRHUPCT,TRAREAPCT,TRAREALANDPCT


In [14]:
df_subset = df[df.STATE == 25][df.COUNTY == 25][['ZCTA5', 'TRACT']]
df_subset.head()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,ZCTA5,TRACT
2196,2108,20101
2197,2108,20302
2198,2108,30300
2199,2108,70101
2200,2109,30100


In [16]:
mapping = df_subset.set_index('TRACT').to_dict()['ZCTA5']
mapping

{100: 2135,
 201: 2135,
 202: 2135,
 301: 2135,
 302: 2135,
 401: 2135,
 402: 2135,
 502: 2467,
 503: 2135,
 504: 2135,
 601: 2135,
 602: 2135,
 701: 2135,
 703: 2215,
 704: 2134,
 802: 2215,
 803: 2215,
 10103: 2215,
 10104: 2215,
 10203: 2215,
 10204: 2215,
 10300: 2215,
 10403: 2215,
 10404: 2115,
 10405: 2215,
 10408: 2215,
 10500: 2116,
 10600: 2199,
 10701: 2116,
 10702: 2116,
 10801: 2116,
 10802: 2116,
 20101: 2114,
 20200: 2114,
 20301: 2114,
 20302: 2114,
 20303: 2114,
 30100: 2113,
 30200: 2114,
 30300: 2203,
 30400: 2113,
 30500: 2113,
 40100: 2129,
 40200: 2129,
 40300: 2129,
 40401: 2129,
 40600: 2129,
 40801: 2129,
 50101: 2128,
 50200: 2128,
 50300: 2128,
 50400: 2128,
 50500: 2128,
 50600: 2128,
 50700: 2128,
 50901: 2128,
 51000: 2128,
 51101: 2128,
 51200: 2128,
 60101: 2127,
 60200: 2127,
 60301: 2127,
 60400: 2127,
 60501: 2127,
 60600: 2210,
 60700: 2127,
 60800: 2127,
 61000: 2127,
 61101: 2127,
 61200: 2210,
 70101: 2210,
 70200: 2116,
 70300: 2116,
 70402: 2118

idk why, but apparently the Census mapping file is missing some Census tracts, like for the Boston Common. I will manually add to the mapping dict. This seems to be the only one missing for my dataset.

In [43]:
mapping[981700] = 2108

## Now to incorporate

In [17]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols.pkl')
df.shape

(786867, 36)

In [47]:
df['zipcode'] = df.tract_and_block_group.map(lambda x: mapping[int(x[:-1])])

### Sanity checking

In [51]:
(df.zipcode == df.LOCATION_ZIPCODE).sum()

442423

In [50]:
(df.zipcode != df.LOCATION_ZIPCODE).sum()

344444

In [53]:
df.LOCATION_ZIPCODE.isnull().sum() # these are all subsets of != since all Census block groups map to zips

182819

In [56]:
df[df.LOCATION_ZIPCODE.notnull()][df.zipcode != df.LOCATION_ZIPCODE].shape[0]

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


161625

In [57]:
161625 + 182819

344444

The 182819 are most interesting. They are not null in the original dataset, and they are different in the new dataset. Could it be different definitions btwn the city's zips and Census ZCTAs? Are the differing zips adjacent to each other, meaning that the Census block groups don't align with the city's zips?

In [61]:
df.head(1).T

,905223
CASE_ENQUIRY_ID,101001983804
OPEN_DT,2017-01-08 12:00:16
TARGET_DT,2017-01-10 08:30:00
CLOSED_DT,NaT
CASE_TITLE,Parking Enforcement
SUBJECT,Transportation - Traffic Division
REASON,Enforcement & Abandoned Vehicles
TYPE,Parking Enforcement
Department,BTDT
SubmittedPhoto,False


In [64]:
df[df.LOCATION_ZIPCODE.notnull()][df.zipcode != df.LOCATION_ZIPCODE].head()[['LOCATION_ZIPCODE', 'zipcode', 'LOCATION_STREET_NAME', 'LATITUDE', 'LONGITUDE', 'tract_and_block_group']]

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,LOCATION_ZIPCODE,zipcode,LOCATION_STREET_NAME,LATITUDE,LONGITUDE,tract_and_block_group
905604,2124.0,2126,7 Thetford Ave,42.2859,-71.0795,1003001
905491,2110.0,2210,500 Atlantic Ave,42.3541,-71.0523,0701011
905647,2121.0,2125,39 Lawrence Ave,42.3123,-71.0780,0903003
905382,2119.0,2130,30 Millmont St,42.3279,-71.0909,0814001
905481,2120.0,2130,806 Parker St,42.3285,-71.0989,0814003


Since from this sample the city's zipcode is always right, I will only impute the ZCTA zip codes which are missing.

I could use Google Maps geocoding (or another service), but I'm limited to 2,500 requests/day.

Another solution, to not have to use the ZCTA zip codes, is to do KNN. That sounds fun, and should be quick. Let's do it!

## Imputing zips via KNN

In [77]:
X = df[df.LOCATION_ZIPCODE.notnull()][['LATITUDE', 'LONGITUDE']]
y = df[df.LOCATION_ZIPCODE.notnull()].LOCATION_ZIPCODE

In [78]:
X_desired = df[df.LOCATION_ZIPCODE.isnull()][['LATITUDE', 'LONGITUDE']]

In [79]:
X.shape, X_desired.shape

((604048, 2), (182819, 2))

In [80]:
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [82]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2
)

In [117]:
NEIGHBORS = 5
clf = KNeighborsClassifier(n_neighbors=NEIGHBORS)
clf.fit(X_train, y_train);

In [118]:
y_predicted = clf.predict(X_test)

In [119]:
precision_score(y_test, y_predicted, average='micro'), \
    recall_score(y_test, y_predicted, average='micro'), \
    accuracy_score(y_test, y_predicted)

(0.96464696631073588, 0.96464696631073588, 0.96464696631073588)

In [120]:
print '{} are expected to be wrong w neighbors={}'.format((1 - accuracy_score(y_test, y_predicted)) * 182819, NEIGHBORS)

6463.20626604 are expected to be wrong w neighbors=5


In [110]:
accuracy_score(df[df.LOCATION_ZIPCODE.notnull()].LOCATION_ZIPCODE, df[df.LOCATION_ZIPCODE.notnull()].zipcode)

0.73243020422218097

Having manually tested the above with neighbors of 1, 2, and 10, there is not much difference. It performs better than the ZCTA zips.

In [121]:
y_desired = clf.predict(X_desired)

In [130]:
X_desired.shape

(182819, 2)

In [138]:
X_desired['zip_imputed'] = y_desired
X_desired.shape

(182819, 3)

In [143]:
mapping = X_desired.set_index(['LATITUDE', 'LONGITUDE']).to_dict()['zip_imputed']
mapping

{(42.3307, -71.0814): 2119.0,
 (42.3765, -71.0568): 2129.0,
 (42.2899, -71.0631): 2124.0,
 (42.2562, -71.1116): 2136.0,
 (42.2828, -71.1321): 2131.0,
 (42.2907, -71.0504): 2122.0,
 (42.3232, -71.1083): 2130.0,
 (42.3447, -71.1419): 2135.0,
 (42.2801, -71.1441): 2132.0,
 (42.3548, -71.1584): 2135.0,
 (42.2963, -71.081): 2124.0,
 (42.3299, -71.0723): 2119.0,
 (42.3413, -71.0696): 2118.0,
 (42.2773, -71.1293): 2131.0,
 (42.3318, -71.1005): 2120.0,
 (42.3196, -71.0853): 2119.0,
 (42.3404, -71.0734): 2118.0,
 (42.3416, -71.053): 2127.0,
 (42.2807, -71.0549): 2124.0,
 (42.3427, -71.0517): 2127.0,
 (42.3365, -71.0381): 2127.0,
 (42.2862, -71.1259): 2131.0,
 (42.3191, -71.0997): 2119.0,
 (42.3526, -71.1581): 2135.0,
 (42.3582, -71.1427): 2135.0,
 (42.3026, -71.0801): 2121.0,
 (42.2813, -71.0838): 2124.0,
 (42.3728, -71.0641): 2129.0,
 (42.3334, -71.0806): 2118.0,
 (42.3278, -71.087): 2119.0,
 (42.3217, -71.0985): 2119.0,
 (42.2815, -71.0607): 2124.0,
 (42.2684, -71.1151): 2136.0,
 (42.3209, -7

In [157]:
mapping[(42.3594, -71.0587)]

2124.0

## OK, now let's map

In [176]:
df['zipcode'] = pd.np.NaN

In [189]:
def get_zipcode(row, d):
    row['zipcode'] = d[(row['LATITUDE'], row['LONGITUDE'])]
    return row

In [191]:
df[['LATITUDE', 'LONGITUDE', 'LOCATION_ZIPCODE']][df.LOCATION_ZIPCODE.isnull()].head() \
    .apply(lambda row: get_zipcode(row, mapping), axis=1)

,LATITUDE,LONGITUDE,LOCATION_ZIPCODE,zipcode
905649,42.3594,-71.0587,NaN,2124.0
905313,42.3594,-71.0587,NaN,2124.0
905648,42.3594,-71.0587,NaN,2124.0
905442,42.3594,-71.0587,NaN,2124.0
905524,42.3594,-71.0587,NaN,2124.0


In [192]:
# http://stackoverflow.com/questions/12307099/modifying-a-subset-of-rows-in-a-pandas-dataframe
df.ix[df.LOCATION_ZIPCODE.isnull(), 'zipcode'] = df[['LATITUDE', 'LONGITUDE', 'LOCATION_ZIPCODE']][df.LOCATION_ZIPCODE.isnull()] \
    .apply(lambda row: get_zipcode(row, mapping), axis=1)

## Sanity check

In [193]:
df[['LOCATION_ZIPCODE', 'zipcode']].head(200).tail()

,LOCATION_ZIPCODE,zipcode
905305,2127.0,NaN
905345,2128.0,NaN
905376,2127.0,NaN
905551,NaN,2124.0
905537,2124.0,NaN


## Filling in `zipcode` and removing `LOCATION_ZIPCODE`

In [196]:
df.zipcode = df.zipcode.fillna(df.LOCATION_ZIPCODE)

In [197]:
df[['LOCATION_ZIPCODE', 'zipcode']].head(200).tail()

,LOCATION_ZIPCODE,zipcode
905305,2127.0,2127.0
905345,2128.0,2128.0
905376,2127.0,2127.0
905551,NaN,2124.0
905537,2124.0,2124.0


In [198]:
df = df.drop('LOCATION_ZIPCODE', axis=1)

## Let's write to file

In [199]:
df.to_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips.pkl')